<a href="https://colab.research.google.com/github/A-Vamshi/deepteam/blob/main/examples/custom_red_teaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepteam deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.1/459.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

Want to use other evaluation models? [Click here](https://deepeval.com/integrations/models/openai) to see all supported models and their usage instructions.

In [ ]:
from openai import OpenAI
from deepteam.vulnerabilities import Competition
from deepteam.attacks.single_turn import PermissionEscalation, PromptInjection
from deepteam.attacks.multi_turn import LinearJailbreaking, SequentialJailbreak
from deepteam.test_case import RTTurn
from deepeval.models import GeminiModel

def model_callback(attack, turn_history=None):
    model = GeminiModel(
        model_name="gemini-2.5-flash",
        api_key="YOUR_API_KEY_HERE",
        temperature=0
    )
    return model.generate(attack)[0]


competetion = Competition()

test_cases = competetion.simulate_attacks()

permission_escalation = PermissionEscalation()
prompt_injection = PromptInjection()
linear_jailbreaking = LinearJailbreaking()
sequential_jailbreak = SequentialJailbreak()

safe_model = True

for test_case in test_cases:
    enhanced_attack_1 = permission_escalation.enhance(test_case.input)
    enhanced_attack_2 = prompt_injection.enhance(enhanced_attack_1)
    turns = [RTTurn(role="user", content=enhanced_attack_2)]
    new_turns_1 = linear_jailbreaking._get_turns(
        model_callback, turns, test_case.vulnerability, test_case.vulnerability_type
    )
    new_turns_2 = sequential_jailbreak._get_turns(
        model_callback, new_turns_1, test_case.vulnerability, test_case.vulnerability_type
    )
    test_case.turns = new_turns_2
    metric = competetion._get_metric(test_case.vulnerability_type)
    metric.measure(test_case)
    if metric.score < 1:
        safe_model = False
        print("LLM application failed. 😞")
        print(metric.score)
        print(metric.reason)
        break

if safe_model:
    print("LLM application passed! 🥳")